In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
import os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import math
import matplotlib.pyplot as plt
import h5py

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
train_dir = #refer to data preprocessing script to set this path
test_dir =  #refer to data preprocessing script to set this path

In [ ]:
train_datagen = ImageDataGenerator(
                rescale = 1./255, 
                shear_range = 0.2, 
                horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
validate_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
#hyper parameters
batch_size = 32
kernel_size = (2, 2) #wrong move
filters = 64
epochs = 20

In [ ]:
train_set = train_datagen.flow_from_directory(train_dir, 
                                              batch_size = batch_size,
                                              target_size = (100, 100),
                                              color_mode = 'rgb',
                                              class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_dir, 
                                            batch_size = batch_size,
                                            target_size = (100, 100),
                                            color_mode = 'rgb',
                                            class_mode = 'categorical')

test_set = validate_datagen.flow_from_directory(validate_dir, 
                                                batch_size = batch_size,
                                                target_size = (100, 100),
                                                color_mode = 'rgb',
                                                class_mode = 'categorical')

In [ ]:
# model architecture
def create_model(model):
    #convolutional feedforwards
    model.add(Conv2D(filters = filters, kernel_size = kernel_size, input_shape = (100, 100, 3), activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = filters, kernel_size = kernel_size, activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = filters, kernel_size = kernel_size, activation = 'sigmoid'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = filters, kernel_size = kernel_size, activation = 'sigmoid'))
    model.add(MaxPool2D(pool_size = (2, 2)))

    #flatten and connect it to the ANN
    model.add(Flatten())
    model.add(Dense(units = 1200, activation = 'relu'))
    model.add(Dense(units = 1000, activation = 'relu'))
    model.add(Dense(units = 500, activation = 'relu'))
    model.add(Dense(units = 400, activation = 'relu'))
    model.add(Dense(units = 300, activation = 'relu'))
    model.add(Dense(units = 120, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'adam',
             metrics = ['accuracy'])
    
    return model

In [ ]:
model =  Sequential()
model = create_model(model)
model.summary()

In [ ]:
#added features for accurate and optimized training
# a check point callback to save our best weights
filepath = #set path
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=False, 
                             mode='max', 
                             save_weights_only=False)

# a reducing lr callback to reduce lr when val_loss doesn't increase
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-7)

# for early stop
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience = 10)

In [ ]:
history = model.fit(train_set, 
          epochs = epochs, 
          validation_data = test_set,
          validation_steps = 479,
          verbose = 1, 
          steps_per_epoch = 2009, 
          callbacks=[checkpoint, reduce_lr, early_stop])

In [ ]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [ ]:
show_final_history(history)

from PIL import Image
import cv2
import numpy as np

img = cv2.imread('beagle.jpg')
res = cv2.resize(img, dsize=(100, 100))
res = res/255.0
res = np.asarray(res)
res = res.reshape((-1, 100, 100, 3))
y_prob = model.predict(res) 
index = np.argmax(y_prob)
labels = {value: key for key, value in train_set.class_indices.items()}
print(labels[index])

In [ ]:
model_score = model.evaluate(test_set)
print("Model Test Loss:",model_score[0])
print("Model Test Accuracy:",model_score[1])